In [1]:
from fcgb.chatbots.chatbot import BaseChatBot
from typing import Dict, Any, List, Annotated
from pydantic import BaseModel
from langgraph.constants import Send
from operator import add
from fcgb.types.research import SimpleTaskModel
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import SystemMessage, HumanMessage

In [2]:
class SimpleTaskDistributionChatBot(BaseChatBot):
    """
    A chatbot that verifies the output of a task.
    """

    def __init__(
            self, 
            llm,
            initial_messages_spec,
            internal_messages_spec,
            memory,
            global_inputs={},
            init_values={},
            prompt_manager_spec = {}
        ):

        super().__init__(
            llm=llm,
            initial_messages_spec=initial_messages_spec,
            internal_messages_spec=internal_messages_spec,
            memory=memory,
            init_values=init_values,
            prompt_manager_spec=prompt_manager_spec,
            global_inputs=global_inputs,
            compile=False
            )

        self.compile_graph()

    def _set_state_class(self):

        task_list_model = self.internal_messages_spec['task_list']['answer_format']
        task_answer_model = self.internal_messages_spec['task']['answer_format']
        answer_model = self.internal_messages_spec['answer']['answer_format']

        class VerificationState(BaseModel):
            parent_thread_id: str 
            template_inputs: Dict[str, str]
            task_list: task_list_model | None # type: ignore
            simple_task_response: Annotated[List[task_answer_model], add] # type: ignore
            verified_answer: answer_model | None # type: ignore

        self.state_class = VerificationState

    def _set_tasks_gen_func(self):
        """
        Set the function that generates the verification tasks.
        """
        
        def tasks_gen_func(state: self.state_class) -> Dict: # type: ignore
            
            task_list = self._invoke_internal_msg(name='task_list', template_inputs=state.template_inputs)

            return {'task_list': task_list}
        
        return tasks_gen_func
        
    def _set_tasks_router_func(self):
        """
        Set the function that distributes tasks along nodes.
        """
        
        def tasks_router_func(state: self.state_class) -> SimpleTaskModel: # type: ignore

            return [Send('task_node',
                        {
                        'template_inputs': state.template_inputs,
                        'task': task,
                        'simple_task_response': None
                        }) for task in state.task_list.tasks]
        
        return tasks_router_func
    
    def _set_task_node_func(self):
        """
        Set the function that solve the tasks.
        """
        
        def task_node_func(state: SimpleTaskModel) -> Dict:

            system = self.internal_prompts['task']['prompt'].format(**state['template_inputs'], **self.global_inputs)
            prompt = state['task'].format(**state['template_inputs'], **self.global_inputs)

            messages = [SystemMessage(content=system), HumanMessage(content=prompt)]
            task_response = self.llm.with_structured_output(self.internal_prompts['task']['answer_format']).invoke(messages)

            return {'simple_task_response': [task_response]}
        
        return task_node_func
    
    def _set_output_node_func(self):

        def output_node_func(state: self.state_class) -> Dict: # type: ignore

            task_response = '\n'.join([resp.recommendations for resp in state.simple_task_response])

            output = self._invoke_internal_msg(name='answer', template_inputs=state.template_inputs | {'task_response': task_response})

            return {'verified_answer': output}
        
        return output_node_func
    
    def _compile_graph(self):

        task_gen_func = self._set_tasks_gen_func()
        tasks_router_func = self._set_tasks_router_func()
        task_node_func = self._set_task_node_func()
        output_node_func = self._set_output_node_func()

        workflow = StateGraph(self.state_class)
        workflow.add_node('task_gen', task_gen_func)
        workflow.add_node('task_node', task_node_func)
        workflow.add_node('output_node', output_node_func)

        workflow.add_edge(START, 'task_gen')
        workflow.add_conditional_edges('task_gen', tasks_router_func, ['task_node'])
        workflow.add_edge('task_node', 'output_node')
        workflow.add_edge('output_node', END)

        self.graph = workflow.compile(
            checkpointer=self.memory,
        )

    def run(self, template_inputs: Dict[str, str], thread_id: str, parent_thread_id: str):

        self.init_thread(thread_id=thread_id, template_inputs=template_inputs)

        config = self._get_config(thread_id)

        return self.graph.invoke({'parent_thread_id': parent_thread_id, 'template_inputs': template_inputs}, config=config)

## fake llm test

In [ ]:
self_research_inputs = {
    'template_inputs': {
        'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
        'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.',
    },
    'ssc_thread_id': 'test_2/ssc1',
    'parent_thread_id': 'test_2'
}

In [33]:
from fcgb.chatbots.specbots import SelfConvForStrategySpecBot, StrategizedSelfResearchSpecBot
from fcgb.fake_models import FakeLLM
from langgraph.checkpoint.memory import MemorySaver

fake_llm = FakeLLM()

memory = MemorySaver()
convbot = SelfConvForStrategySpecBot(llm=fake_llm, memory=memory)
strategybot = StrategizedSelfResearchSpecBot(llm=fake_llm, self_conv_bot=convbot, memory=memory)

In [5]:
research_output = strategybot.run(**self_research_inputs)
research_output

{'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting.',
  'context': 'You want to prepare a full list of different CoT prompting methods for further research.'},
 'ssc_thread_id': 'test_2/ssc1',
 'parent_thread_id': 'test_2',
 'ssc_summary': SimpleAnswerModel(answer='Fake string gybaw'),
 'strategies': StrategyTaskModel(strategies=[{'strategy_description': 'Fake string ajvzj', 'paraphrased_task': 'Fake string hidjp', 'paraphrased_context': 'Fake string kcrsz'}, {'strategy_description': 'Fake string acpzv', 'paraphrased_task': 'Fake string fesqu', 'paraphrased_context': 'Fake string szngl'}, {'strategy_description': 'Fake string hifza', 'paraphrased_task': 'Fake string tewjq', 'paraphrased_context': 'Fake string wxjlx'}]),
 'sc_thread_id': ['test_2/ssc1/self_conv0',
  'test_2/ssc1/self_conv1',
  'test_2/ssc1/self_conv2'],
 'sc_summary': ['Fake string dijpp', 'Fake string ivagf', 'Fake string bzvwi']}

In [6]:
verification_inputs ={
    'template_inputs': self_research_inputs['template_inputs'] | {'answer': research_output['ssc_summary'].answer},
    'thread_id': self_research_inputs['ssc_thread_id'] + '/ver',
    'parent_thread_id': self_research_inputs['ssc_thread_id']
}

In [7]:
from fcgb.cfg.chat_inputs_spec import ResearchVerificationConfig

verificationbot = SimpleTaskDistributionChatBot(
    llm=fake_llm,
    initial_messages_spec=ResearchVerificationConfig.initial_messages_spec,
    internal_messages_spec=ResearchVerificationConfig.internal_messages_spec,
    memory=memory,
    global_inputs=ResearchVerificationConfig.global_inputs,
    init_values=ResearchVerificationConfig.init_values,
    prompt_manager_spec=ResearchVerificationConfig.prompt_manager_spec
)

In [8]:
verification_output = verificationbot.run(**verification_inputs)
verification_output

{'parent_thread_id': 'test_2/ssc1',
 'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting.',
  'context': 'You want to prepare a full list of different CoT prompting methods for further research.',
  'answer': 'Fake string gybaw'},
 'task_list': PromptTemplatesListModel(analysis='Fake string vfksh', tasks=['Fake string jiukt', 'Fake string gvsbb']),
 'simple_task_response': [SingleVerificationModel(analysis='Fake string gcynr', recommendations='Fake string ecmkp'),
  SingleVerificationModel(analysis='Fake string mltxy', recommendations='Fake string tydmj')],
 'verified_answer': SimpleAnswerModel(answer='Fake string xapnf')}

## real llm test

In [9]:
from fcgb.cfg.precompiled import get_checkpointer, get_llm

memory = get_checkpointer(checkpointer_mode='mongodb', mode='test')
llm = get_llm(llm_model='google')

In [34]:
from fcgb.chatbots.specbots import SelfConvForStrategySpecBot, StrategizedSelfResearchSpecBot
from fcgb.cfg.chat_inputs_spec import ResearchVerificationConfig

convbot = SelfConvForStrategySpecBot(llm=llm, memory=memory)
strategybot = StrategizedSelfResearchSpecBot(llm=llm, self_conv_bot=convbot, memory=memory)
verificationbot = SimpleTaskDistributionChatBot(
    llm=llm,
    initial_messages_spec=ResearchVerificationConfig.initial_messages_spec,
    internal_messages_spec=ResearchVerificationConfig.internal_messages_spec,
    memory=memory,
    global_inputs=ResearchVerificationConfig.global_inputs,
    init_values=ResearchVerificationConfig.init_values,
    prompt_manager_spec=ResearchVerificationConfig.prompt_manager_spec
)

In [ ]:
"""from fcgb.cfg.precompiled import get_db_client

db = get_db_client(db_engine='mongodb', mode='test')

db['checkpoints_writes'].delete_many({'thread_id': 'partial/research/cot/ssc1'})
db['checkpoints'].delete_many({'thread_id': 'partial/research/cot/ssc1'})

strategybot.get_state('partial/research/cot/ssc1')"""

In [35]:
self_research_inputs = {
    'template_inputs': {
        'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
        'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.',
    },
    'ssc_thread_id': 'partial/research/cot/ssc4',
    'parent_thread_id': 'partial/research/cot'
}

For a task which is: 'List all variants of Chain-of-Thought (CoT) prompting techniques.'
And a context for it: 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.'

List possible mistakes that LLM can make answering for a given task and context.

In [37]:
research_output = strategybot.run(**self_research_inputs)
research_output

{'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
  'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.'},
 'ssc_thread_id': 'partial/research/cot/ssc4',
 'parent_thread_id': 'partial/research/cot',
 'ssc_summary': SimpleAnswerModel(answer='The variants of Chain-of-Thought (CoT) prompting techniques are:\n\n*   Standard CoT\n*   Few-shot CoT\n*   Zero-shot CoT\n*   Self-Consistency\n*   Tree-of-Thoughts (ToT)\n*   Graph-of-Thoughts (GoT)\n*   Program-aided CoT / Program-of-Thoughts (PoT)\n*   CoT with Knowledge Graphs\n*   Active Prompting with CoT / Active Prompting with Few-shot CoT\n*   Multimodal CoT\n*   Self-Consistency with Tree-of-Thoughts\n*   Program-aided CoT with Knowledge Graphs\n*   Least-to-Most CoT\n*   Chain-of-Verification (CoV)\n*   Step-Back Prompting\n*   Generated Knowledge Prompting'),


In [36]:
research_output = strategybot.get_state(self_research_inputs['ssc_thread_id'])
research_output

In [38]:
for line in research_output['ssc_summary'].answer.split('\n'):
    print(line)

The variants of Chain-of-Thought (CoT) prompting techniques are:

*   Standard CoT
*   Few-shot CoT
*   Zero-shot CoT
*   Self-Consistency
*   Tree-of-Thoughts (ToT)
*   Graph-of-Thoughts (GoT)
*   Program-aided CoT / Program-of-Thoughts (PoT)
*   CoT with Knowledge Graphs
*   Active Prompting with CoT / Active Prompting with Few-shot CoT
*   Multimodal CoT
*   Self-Consistency with Tree-of-Thoughts
*   Program-aided CoT with Knowledge Graphs
*   Least-to-Most CoT
*   Chain-of-Verification (CoV)
*   Step-Back Prompting
*   Generated Knowledge Prompting


In [39]:
verification_inputs ={
    'template_inputs': self_research_inputs['template_inputs'] | {'answer': research_output['ssc_summary'].answer},
    'thread_id': self_research_inputs['ssc_thread_id'] + '/ver1',
    'parent_thread_id': self_research_inputs['ssc_thread_id']
}

In [40]:
verification_output = verificationbot.run(**verification_inputs)
verification_output

{'parent_thread_id': 'partial/research/cot/ssc4',
 'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
  'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.',
  'answer': 'The variants of Chain-of-Thought (CoT) prompting techniques are:\n\n*   Standard CoT\n*   Few-shot CoT\n*   Zero-shot CoT\n*   Self-Consistency\n*   Tree-of-Thoughts (ToT)\n*   Graph-of-Thoughts (GoT)\n*   Program-aided CoT / Program-of-Thoughts (PoT)\n*   CoT with Knowledge Graphs\n*   Active Prompting with CoT / Active Prompting with Few-shot CoT\n*   Multimodal CoT\n*   Self-Consistency with Tree-of-Thoughts\n*   Program-aided CoT with Knowledge Graphs\n*   Least-to-Most CoT\n*   Chain-of-Verification (CoV)\n*   Step-Back Prompting\n*   Generated Knowledge Prompting'},
 'task_list': PromptTemplatesListModel(analysis='The answer provides a l

In [41]:
for line in verification_output['task_list'].analysis.split('\n'):
    print(line, )

The answer provides a list of Chain-of-Thought (CoT) prompting techniques. The task is to list all variants, and the context specifies that this list is for potential LLM users learning to use LLMs efficiently. Therefore, the list should be comprehensive, accurate, and relevant to practical LLM usage. Potential mistakes include: 1) Inaccuracy: Listing techniques that are not actual CoT variants or are misnamed. 2) Redundancy: Listing essentially the same technique under different names. 3) Irrelevance: Including highly theoretical or impractical techniques that are not useful for typical LLM users. 4) Omission: Missing important and widely used CoT variants. 5) Lack of clarity: Not providing enough information to distinguish between similar techniques. 6) Hallucination: Inventing CoT variants that do not exist. 7) Overly specific/niche techniques: Including variants that are only applicable in very narrow scenarios, which might not be helpful for the target audience. 8) Incorrect group

In [42]:
sep = f'\n{'-' * 40}\n'
for sc_summ in verification_output['task_list'].tasks:
    for line in sc_summ.split('\n'):
        print(line)
    print(sep)

Verify the accuracy of each listed technique. Is it a real CoT variant?

----------------------------------------

Check for redundancy. Are any techniques listed multiple times under different names?

----------------------------------------

Assess the relevance of each technique to practical LLM usage. Is it useful for typical users?

----------------------------------------

Identify any omissions. Are there important CoT variants missing from the list?

----------------------------------------

Evaluate the clarity of the descriptions. Can a user easily understand the difference between similar techniques?

----------------------------------------

Check for hallucinations. Are any techniques invented or non-existent?

----------------------------------------

Determine if any techniques are overly specific or niche, making them less useful for the target audience.

----------------------------------------

Verify the correct grouping of techniques. Are all listed items actually C

In [43]:
sep = f'\n{'-' * 40}\n'
for sc_summ in verification_output['simple_task_response']:
    for line in sc_summ.analysis.split('\n'):
        print(line)
    print(sep)

All listed items appear to be valid variants or extensions of the Chain-of-Thought (CoT) prompting technique. They represent different approaches to improve reasoning and problem-solving in large language models.

----------------------------------------

The list seems comprehensive and covers a wide range of CoT variants. However, some entries might be redundant or represent combinations of other techniques. Need to verify if 'Self-Consistency with Tree-of-Thoughts' is simply a combination of 'Self-Consistency' and 'Tree-of-Thoughts'. Similarly, 'Program-aided CoT with Knowledge Graphs' combines 'Program-aided CoT' and 'CoT with Knowledge Graphs'. Also, 'Program-aided CoT' and 'Program-of-Thoughts (PoT)' seem to refer to the same technique.

----------------------------------------

The provided list of Chain-of-Thought (CoT) prompting techniques is comprehensive and includes many relevant variants. The list covers a wide spectrum of CoT approaches, from basic ones like Standard CoT 

In [44]:
sep = f'\n{'-' * 40}\n'
for sc_summ in verification_output['simple_task_response']:
    for line in sc_summ.recommendations.split('\n'):
        print(line)
    print(sep)

No changes are needed. The list seems comprehensive and accurate based on current research and literature on CoT prompting techniques. Each item represents a distinct variation or extension of the original CoT approach, addressing different aspects of reasoning or incorporating external knowledge sources or modalities..

----------------------------------------

1.  Consider removing 'Self-Consistency with Tree-of-Thoughts' if it's adequately covered by discussing 'Self-Consistency' and 'Tree-of-Thoughts' separately.
2.  Consider removing 'Program-aided CoT with Knowledge Graphs' if it's adequately covered by discussing 'Program-aided CoT' and 'CoT with Knowledge Graphs' separately.
3.  Clarify that 'Program-aided CoT' and 'Program-of-Thoughts (PoT)' refer to the same technique and potentially merge them into a single entry.


----------------------------------------

To enhance the answer's usefulness for potential LLM users, I recommend adding a brief note about the practical applica

In [45]:
for line in verification_output['verified_answer'].answer.split('\n'):
    print(line)

Chain-of-Thought (CoT) Prompting Techniques: A Beginner's Guide

This list introduces various Chain-of-Thought (CoT) prompting techniques, designed to enhance the reasoning capabilities of Large Language Models (LLMs). We've organized them from basic to advanced to facilitate progressive learning.

Basic CoT Techniques:

*   Zero-shot CoT: Instruct the model to think step by step without providing any examples. (e.g., "Let's think step by step.")
*   Few-shot CoT: Provide a few examples of questions and their step-by-step reasoning to guide the model.
*   Standard CoT: This is the general application of chain of thought prompting, where you encourage the model to explain its reasoning process.

Intermediate CoT Techniques:

*   Active Prompting with CoT: Selectively choose examples to prompt the model, optimizing for learning and performance.
*   Backward CoT: Start with the answer and ask the model to reason backward to the initial question.
*   Chain of Hindsight: After generating an

For a task: 'List all variants of Chain-of-Thought (CoT) prompting.'
Provide 